# Connect4 AI Assignment

**Team Members:**
- Team Member 1
- Team Member 2
- Team Member 3

## Introduction

This notebook implements and evaluates two AI algorithms for playing the Connect4 game:

1. Monte Carlo Tree Search (MCTS) with UCT
2. ID3 Decision Tree trained on MCTS-generated data

We will evaluate their performance and compare their strengths and weaknesses.


In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys
import time
from IPython.display import display, HTML

# Add the project root to the path for imports
sys.path.append(os.path.dirname(os.getcwd()))

# Import project modules
from game_structure import style as s
from game_structure import game_engine as game
from ai_alg.monte_carlo import monte_carlo, Node
from ai_alg.alpha_beta import alpha_beta
from ai_alg.ID3 import ID3Tree

## 1. Game Implementation

First, let's review the Connect4 game structure implementation and demonstrate how it works.

In [ ]:
# Helper function to display Connect4 board in the notebook
def display_board(board):
    """Display a Connect4 board in the notebook"""
    rows, cols = board.shape
    html = '<table style="border-collapse: collapse; border: 3px solid blue; width: 280px; height: 240px;">' 
    
    for r in range(rows-1, -1, -1):  # Display from top to bottom
        html += '<tr>'
        for c in range(cols):
            color = 'white'
            if board[r, c] == 1:
                color = 'red'
            elif board[r, c] == 2:
                color = 'yellow'
            html += f'<td style="border: 1px solid black; width: 40px; height: 40px; \
                      background-color: {color}; border-radius: 50%;"></td>'
        html += '</tr>'
    
    # Add column numbers
    html += '<tr>'
    for c in range(cols):
        html += f'<td style="text-align: center; font-weight: bold;">{c}</td>'
    html += '</tr>'
    
    html += '</table>'
    return HTML(html)

# Create an empty board
board = np.zeros((s.ROWS, s.COLUMNS), dtype=int)

# Make some example moves
game.drop_piece(board, 0, 3, 1)  # Player 1 in column 3, row 0
game.drop_piece(board, 0, 2, 2)  # Player 2 in column 2, row 0
game.drop_piece(board, 0, 4, 1)  # Player 1 in column 4, row 0
game.drop_piece(board, 1, 3, 2)  # Player 2 in column 3, row 1

# Display the board
display_board(board)

## 2. Monte Carlo Tree Search Implementation

Now let's examine our MCTS implementation and demonstrate how it selects moves.

In [ ]:
# Create board with a potential winning move
board = np.zeros((s.ROWS, s.COLUMNS), dtype=int)

# Set up a position where Player 2 can win with a move
for col, row in [(2, 0), (3, 0), (4, 0)]:
    game.drop_piece(board, row, col, 2)

display_board(board)
print("Player 2 can win by playing column 1 or 5")

# Let's run MCTS and see what it suggests
root = Node(board=board, last_player=s.FIRST_PLAYER_PIECE)
mc = monte_carlo(root)
start_time = time.time()
move = mc.start(2)  # Run MCTS for 2 seconds
elapsed = time.time() - start_time

print(f"MCTS selected column: {move} in {elapsed:.2f} seconds")

### MCTS Exploration Parameter Analysis

Let's analyze how the number of children explored affects MCTS performance.

In [ ]:
# Let's analyze different numbers of selected children
# In our monte_carlo.py, the select_children method limits exploration to 4 children
# Let's modify it temporarily for testing

def test_child_selection(board, max_children_values=[2, 4, 7], search_time=1.0):
    results = []
    
    for max_children in max_children_values:
        # Temporarily modify Node.select_children to use our max_children parameter
        original_select_children = Node.select_children
        
        def modified_select_children(self):
            if len(self.children) > max_children:
                return random.sample(self.children, max_children)
            return self.children
        
        # Apply the monkey patch
        Node.select_children = modified_select_children
        
        # Run MCTS with this configuration
        root = Node(board=board.copy(), last_player=s.FIRST_PLAYER_PIECE)
        mc = monte_carlo(root)
        start_time = time.time()
        move = mc.start(search_time)
        elapsed = time.time() - start_time
        
        # Calculate number of nodes explored
        nodes_explored = count_nodes(root)
        
        results.append({
            'max_children': max_children,
            'move_selected': move,
            'time': elapsed,
            'nodes_explored': nodes_explored
        })
        
        # Restore original method
        Node.select_children = original_select_children
    
    return pd.DataFrame(results)

# Helper to count nodes in the tree
def count_nodes(node):
    if not node.children:
        return 1
    return 1 + sum(count_nodes(child[0]) for child in node.children)

# Test with our board
import random  # For random.sample
results_df = test_child_selection(board)
results_df

In [ ]:
# Let's visualize the results
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.bar(results_df['max_children'].astype(str), results_df['nodes_explored'])
plt.title('Number of Nodes Explored')
plt.xlabel('Max Children per Node')
plt.ylabel('Nodes Explored')

plt.subplot(1, 2, 2)
plt.bar(results_df['max_children'].astype(str), results_df['time'])
plt.title('Computation Time')
plt.xlabel('Max Children per Node')
plt.ylabel('Time (seconds)')

plt.tight_layout()
plt.show()

# Discussion of results
print("Results Analysis:")
print("1. As we increase the number of children explored, we see a significant increase in nodes explored")
print("2. This leads to more thorough search but higher computational costs")
print("3. The optimal value balances exploration breadth with computational efficiency")

## 3. Dataset Generation

Now, let's generate a dataset of Connect4 states and moves using MCTS. For demonstration purposes, we'll generate a small dataset in the notebook.

In [ ]:
# Generate a small dataset for demonstration
def generate_demo_dataset(n_games=5, search_time=1.0):
    all_records = []
    
    for game_idx in range(n_games):
        print(f"Generating game {game_idx + 1}/{n_games}...")
        board = np.zeros((s.ROWS, s.COLUMNS), dtype=int)
        game_records = []
        turn = s.FIRST_PLAYER_PIECE
        
        while True:
            # Record current state
            state = board.flatten().tolist()
            
            # Get move from MCTS
            last_player = s.SECOND_PLAYER_PIECE if turn == s.FIRST_PLAYER_PIECE else s.FIRST_PLAYER_PIECE
            root = Node(board=board.copy(), last_player=last_player)
            mc = monte_carlo(root)
            move = mc.start(search_time)
            
            # Record state and chosen move
            game_records.append(state + [move])
            
            # Make the move
            row = game.get_next_open_row(board, move)
            game.drop_piece(board, row, move, turn)
            
            # Check if game is over
            if game.winning_move(board, turn) or game.is_game_tied(board):
                print(f"Game over after {len(game_records)} moves")
                print("Final board:")
                display(display_board(board))
                break
            
            # Switch players
            turn = s.SECOND_PLAYER_PIECE if turn == s.FIRST_PLAYER_PIECE else s.FIRST_PLAYER_PIECE
        
        all_records.extend(game_records)
    
    # Create DataFrame
    columns = [f"cell_{i}" for i in range(42)] + ["move"]
    df = pd.DataFrame(all_records, columns=columns)
    return df

# Generate a small demo dataset
demo_dataset = generate_demo_dataset(n_games=2, search_time=0.5)
print(f"Generated dataset with {len(demo_dataset)} state-move pairs")
demo_dataset.head()

### Load Full Dataset

In practice, we would generate a larger dataset using our script and load it here.

In [ ]:
# Check if the full dataset exists, otherwise use our demo dataset
dataset_path = os.path.join("data", "connect4_dataset.csv")
if os.path.exists(dataset_path):
    print(f"Loading dataset from {dataset_path}")
    dataset = pd.read_csv(dataset_path)
else:
    print("Full dataset not found, using demo dataset")
    dataset = demo_dataset
    
print(f"Dataset shape: {dataset.shape}")

# Analyze move distribution
plt.figure(figsize=(10, 5))
move_counts = dataset['move'].value_counts().sort_index()
plt.bar(move_counts.index, move_counts.values)
plt.title('Distribution of Moves in Dataset')
plt.xlabel('Column')
plt.ylabel('Count')
plt.xticks(range(7))
plt.grid(axis='y', alpha=0.3)
plt.show()

print("Move distribution (%):\n")
move_percentage = move_counts / len(dataset) * 100
for col, pct in move_percentage.items():
    print(f"Column {col}: {pct:.2f}%")

## 4. ID3 Decision Tree Implementation and Training

Now, let's train our ID3 decision tree on the Connect4 dataset.

In [ ]:
# Split dataset into training and testing sets
def manual_train_test_split(X, y, test_size=0.2, random_seed=42):
    """Split arrays into random train and test subsets"""
    np.random.seed(random_seed)
    indices = np.random.permutation(len(X))
    test_size = int(len(X) * test_size)
    test_indices = indices[:test_size]
    train_indices = indices[test_size:]
    
    X_train = X.iloc[train_indices] if isinstance(X, pd.DataFrame) else X[train_indices]
    X_test = X.iloc[test_indices] if isinstance(X, pd.DataFrame) else X[test_indices]
    y_train = y.iloc[train_indices] if isinstance(y, pd.Series) else y[train_indices]
    y_test = y.iloc[test_indices] if isinstance(y, pd.Series) else y[test_indices]
    
    return X_train, X_test, y_train, y_test

# Prepare the data
X = dataset.iloc[:, :-1]
y = dataset.iloc[:, -1]

X_train, X_test, y_train, y_test = manual_train_test_split(X, y, test_size=0.2)
print(f"Training set size: {len(X_train)}")
print(f"Testing set size: {len(X_test)}")

# Train the ID3 decision tree
print("Training ID3 decision tree...")
start_time = time.time()
id3_tree = ID3Tree(max_depth=10)  # Limiting depth for reasonable training time
id3_tree.fit(X_train, y_train)
print(f"Training completed in {time.time() - start_time:.2f} seconds")

# Evaluate the model
y_pred = id3_tree.predict(X_test)
accuracy = np.mean(y_pred == y_test) * 100
print(f"ID3 accuracy on test set: {accuracy:.2f}%")

### Visualize Decision Tree Structure

Let's examine a simplified version of the learned decision tree.

In [ ]:
# Helper function to visualize a simplified version of the tree
def visualize_tree(tree, max_depth=3, depth=0, prefix=""):
    """Print a simplified visualization of the decision tree"""
    if depth > max_depth:
        return "..."  # Too deep, truncate
    
    if not isinstance(tree, dict):
        return f"→ Column {tree}"  # Leaf node (predicted move)
    
    # Get the attribute and its value
    attr, branches = list(tree.items())[0]
    feature, value = attr
    
    # Convert feature name to board position for better interpretation
    feature_idx = int(feature.replace("cell_", ""))
    row = feature_idx // s.COLUMNS
    col = feature_idx % s.COLUMNS
    
    result = f"If cell({row},{col}) == {value}:\n"
    result += f"{prefix}├─ Yes: {visualize_tree(branches['left'], max_depth, depth+1, prefix+'│  ')}\n"
    result += f"{prefix}└─ No: {visualize_tree(branches['right'], max_depth, depth+1, prefix+'   ')}"
    return result

# Visualize the tree
print("Simplified Decision Tree Visualization (limited to depth 3):")
print(visualize_tree(id3_tree.tree, max_depth=3))

## 5. Compare Algorithm Performance

Let's compare the performance of our different algorithms: MCTS, Alpha-Beta, and ID3.

In [ ]:
def compare_algorithms(board, time_limit=1.0):
    """Compare different algorithms on the same board position"""
    results = {}
    
    # 1. Monte Carlo Tree Search
    start_time = time.time()
    root = Node(board=board.copy(), last_player=s.FIRST_PLAYER_PIECE)
    mc = monte_carlo(root)
    mcts_move = mc.start(time_limit)
    mcts_time = time.time() - start_time
    results['MCTS'] = {'move': mcts_move, 'time': mcts_time}
    
    # 2. Alpha-Beta Pruning
    start_time = time.time()
    ab_move = alpha_beta(board.copy())
    ab_time = time.time() - start_time
    results['Alpha-Beta'] = {'move': ab_move, 'time': ab_time}
    
    # 3. ID3 Decision Tree
    start_time = time.time()
    # Convert board to feature vector
    features = pd.DataFrame([board.flatten().tolist()], columns=[f'cell_{i}' for i in range(42)])
    id3_move = id3_tree.predict(features).iloc[0]
    id3_time = time.time() - start_time
    results['ID3'] = {'move': id3_move, 'time': id3_time}
    
    return results

# Create a few test positions
test_positions = []

# Position 1: Empty board
board1 = np.zeros((s.ROWS, s.COLUMNS), dtype=int)
test_positions.append(("Empty board", board1))

# Position 2: Mid-game position
board2 = np.zeros((s.ROWS, s.COLUMNS), dtype=int)
moves = [(0, 3, 1), (0, 2, 2), (0, 4, 1), (0, 1, 2), (1, 3, 1), (0, 0, 2)]
for r, c, p in moves:
    game.drop_piece(board2, r, c, p)
test_positions.append(("Mid-game position", board2))

# Position 3: Near-win position
board3 = np.zeros((s.ROWS, s.COLUMNS), dtype=int)
moves = [(0, 0, 1), (0, 1, 1), (0, 2, 1), (0, 6, 2), (1, 6, 2), (2, 6, 2)]
for r, c, p in moves:
    game.drop_piece(board3, r, c, p)
test_positions.append(("Near-win position", board3))

# Compare algorithms on each position
all_results = []

for name, board in test_positions:
    print(f"\nPosition: {name}")
    display(display_board(board))
    
    results = compare_algorithms(board)
    
    print("Algorithm comparison:")
    for algo, data in results.items():
        print(f"{algo}: chose column {data['move']} in {data['time']*1000:.2f} ms")
    
    # Add to results for later analysis
    for algo, data in results.items():
        all_results.append({
            'position': name,
            'algorithm': algo,
            'move': data['move'],
            'time_ms': data['time'] * 1000
        })

# Convert to DataFrame
results_df = pd.DataFrame(all_results)

# Visualize time comparison
plt.figure(figsize=(10, 6))
positions = results_df['position'].unique()
algorithms = results_df['algorithm'].unique()
x = np.arange(len(positions))
width = 0.25

for i, algo in enumerate(algorithms):
    algo_data = results_df[results_df['algorithm'] == algo]
    plt.bar(x + i*width - width, algo_data['time_ms'], width, label=algo)

plt.title('Algorithm Performance Comparison')
plt.xlabel('Position')
plt.ylabel('Time (ms)')
plt.xticks(x, positions)
plt.legend()
plt.yscale('log')  # Log scale to show large differences
plt.tight_layout()
plt.show()

## 6. Discussion and Conclusion

### Algorithm Comparison

Let's summarize what we've learned about each algorithm:

#### Monte Carlo Tree Search (MCTS)
- **Strengths**: Explores the game tree effectively, doesn't require a specific heuristic evaluation function, and can be tuned by adjusting exploration parameters.
- **Weaknesses**: Computationally expensive, performance depends on available search time.
- **Best use case**: When high-quality play is needed and computational resources are available.

#### Alpha-Beta Pruning
- **Strengths**: Can search deeper than MCTS in the same time budget by pruning unpromising branches.
- **Weaknesses**: Heavily dependent on the quality of the evaluation function.
- **Best use case**: When a good evaluation function is available and consistent decision-making is important.

#### ID3 Decision Tree
- **Strengths**: Extremely fast execution time, learns patterns from training data automatically.
- **Weaknesses**: Quality entirely depends on training data, doesn't adapt to new situations.
- **Best use case**: When execution speed is critical or as part of a hybrid system.

### Practical Recommendations

Based on our experiments, we recommend:

1. Use ID3 for real-time play where decisions must be made quickly
2. Use MCTS for higher quality play when computation time is available
3. Consider a hybrid approach: use ID3 for opening moves and early/mid-game, then switch to MCTS for critical endgame situations

### Future Work

Potential improvements for future development:

1. Implement a hybrid algorithm that combines the strengths of multiple approaches
2. Improve the training dataset quality by using stronger MCTS parameters
3. Explore feature engineering to improve the ID3 model's performance
4. Implement a neural network approach as an alternative learning method

## 7. Running a Full Connect4 Game

For completeness, let's run a full game between two AI algorithms.

In [ ]:
def run_ai_vs_ai_game(player1_algo='MCTS', player2_algo='ID3', search_time=1.0):
    """Run a game between two AI algorithms"""
    board = np.zeros((s.ROWS, s.COLUMNS), dtype=int)
    moves_history = []
    
    # Current player (1 or 2)
    turn = s.FIRST_PLAYER_PIECE
    
    # Map algorithm names to functions
    algo_funcs = {
        'MCTS': lambda b: monte_carlo(Node(b.copy(), s.SECOND_PLAYER_PIECE if turn == s.FIRST_PLAYER_PIECE else s.FIRST_PLAYER_PIECE)).start(search_time),
        'Alpha-Beta': lambda b: alpha_beta(b.copy()),
        'ID3': lambda b: id3_tree.predict(pd.DataFrame([b.flatten().tolist()], columns=[f'cell_{i}' for i in range(42)])).iloc[0]
    }
    
    print(f"Starting game: {player1_algo} (Player 1) vs {player2_algo} (Player 2)")
    display(display_board(board))
    
    # Game loop
    while True:
        # Determine current algorithm
        current_algo = player1_algo if turn == s.FIRST_PLAYER_PIECE else player2_algo
        
        # Get move from appropriate algorithm
        start_time = time.time()
        move = algo_funcs[current_algo](board)
        elapsed = time.time() - start_time
        
        print(f"Player {turn} ({current_algo}) chose column {move} in {elapsed*1000:.2f} ms")
        
        # Make the move
        row = game.get_next_open_row(board, move)
        game.drop_piece(board, row, move, turn)
        moves_history.append((turn, move))
        
        # Display updated board
        display(display_board(board))
        
        # Check if game is over
        if game.winning_move(board, turn):
            print(f"Player {turn} ({current_algo}) wins after {len(moves_history)} moves!")
            break
        
        if game.is_game_tied(board):
            print(f"Game is tied after {len(moves_history)} moves!")
            break
        
        # Switch players
        turn = s.SECOND_PLAYER_PIECE if turn == s.FIRST_PLAYER_PIECE else s.FIRST_PLAYER_PIECE
    
    return board, moves_history

# Run a game between MCTS and ID3
final_board, moves = run_ai_vs_ai_game(player1_algo='MCTS', player2_algo='ID3', search_time=0.5)